# Projekt ML - Przewidywanie Intencji Zakupowych Online Shoppers

## Cel Projektu
Klasyfikacja binarna przewidująca, czy użytkownik dokona zakupu (kolumna `Revenue`) na podstawie danych o zachowaniu na stronie e-commerce.

## Uwaga
Klasy OOP są zdefiniowane w tym notebooku. Alternatywnie można zaimportować klasy z pliku `ml_classes.py`:
```python
from ml_classes import DataLoader, DataPreprocessor, DataAnalyzer, FeatureEngineer, ModelTrainer, HyperparameterTuner
```
Plik `ml_classes.py` jest używany przez testy jednostkowe.


## 1. Import bibliotek


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, classification_report
import warnings
warnings.filterwarnings('ignore')

# Konfiguracja wizualizacji
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline


## 2. Definicja klas OOP


In [ ]:
class DataLoader:
    """Klasa odpowiedzialna za wczytanie danych z pliku CSV"""
    
    def __init__(self):
        self.data = None
        self.path = None
    
    def load_data(self, path: str) -> pd.DataFrame:
        """
        Wczytuje dane z pliku CSV
        
        Parameters:
        -----------
        path : str
            Ścieżka do pliku CSV
            
        Returns:
        --------
        pd.DataFrame
            Wczytane dane
        """
        try:
            self.data = pd.read_csv(path)
            self.path = path
            print(f"Dane wczytane pomyślnie z: {path}")
            print(f"Kształt danych: {self.data.shape}")
            return self.data
        except Exception as e:
            print(f"Błąd podczas wczytania danych: {e}")
            return None
    
    def get_info(self) -> dict:
        """
        Zwraca podstawowe informacje o zbiorze danych
        
        Returns:
        --------
        dict
            Słownik z informacjami o danych
        """
        if self.data is None:
            return {"error": "Dane nie zostały wczytane"}
        
        info = {
            "shape": self.data.shape,
            "columns": list(self.data.columns),
            "dtypes": self.data.dtypes.to_dict(),
            "missing_values": self.data.isnull().sum().to_dict(),
            "memory_usage": self.data.memory_usage(deep=True).sum()
        }
        return info


In [ ]:
class DataPreprocessor:
    """Klasa odpowiedzialna za preprocessing danych"""
    
    def __init__(self):
        self.label_encoders = {}
        self.scaler = StandardScaler()
        self.is_fitted = False
    
    def handle_missing_values(self, df: pd.DataFrame, strategy: str = 'drop') -> pd.DataFrame:
        """
        Obsługuje brakujące wartości
        
        Parameters:
        -----------
        df : pd.DataFrame
            DataFrame do przetworzenia
        strategy : str
            Strategia obsługi ('drop', 'mean', 'median', 'mode')
            
        Returns:
        --------
        pd.DataFrame
            DataFrame po obsłudze brakujących wartości
        """
        df_processed = df.copy()
        
        if strategy == 'drop':
            df_processed = df_processed.dropna()
        elif strategy == 'mean':
            numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
            df_processed[numeric_cols] = df_processed[numeric_cols].fillna(df_processed[numeric_cols].mean())
        elif strategy == 'median':
            numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
            df_processed[numeric_cols] = df_processed[numeric_cols].fillna(df_processed[numeric_cols].median())
        elif strategy == 'mode':
            for col in df_processed.columns:
                df_processed[col] = df_processed[col].fillna(df_processed[col].mode()[0] if not df_processed[col].mode().empty else 0)
        
        return df_processed
    
    def encode_categorical(self, df: pd.DataFrame, columns: list = None, method: str = 'label') -> pd.DataFrame:
        """
        Koduje zmienne kategoryczne
        
        Parameters:
        -----------
        df : pd.DataFrame
            DataFrame do przetworzenia
        columns : list
            Lista kolumn do zakodowania (None = automatyczne wykrycie)
        method : str
            Metoda kodowania ('label' lub 'onehot')
            
        Returns:
        --------
        pd.DataFrame
            DataFrame z zakodowanymi zmiennymi
        """
        df_encoded = df.copy()
        
        if columns is None:
            # Automatyczne wykrycie kolumn kategorycznych
            categorical_cols = df_encoded.select_dtypes(include=['object', 'bool']).columns.tolist()
        else:
            categorical_cols = columns
        
        if method == 'label':
            for col in categorical_cols:
                if col in df_encoded.columns:
                    le = LabelEncoder()
                    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
                    self.label_encoders[col] = le
        elif method == 'onehot':
            df_encoded = pd.get_dummies(df_encoded, columns=categorical_cols, prefix=categorical_cols)
        
        return df_encoded
    
    def normalize_features(self, df: pd.DataFrame, columns: list = None, fit: bool = True) -> pd.DataFrame:
        """
        Normalizuje zmienne numeryczne
        
        Parameters:
        -----------
        df : pd.DataFrame
            DataFrame do przetworzenia
        columns : list
            Lista kolumn do normalizacji (None = wszystkie numeryczne)
        fit : bool
            Czy dopasować scaler (True dla train, False dla test)
            
        Returns:
        --------
        pd.DataFrame
            DataFrame z znormalizowanymi zmiennymi
        """
        df_normalized = df.copy()
        
        if columns is None:
            numeric_cols = df_normalized.select_dtypes(include=[np.number]).columns.tolist()
        else:
            numeric_cols = columns
        
        if fit:
            df_normalized[numeric_cols] = self.scaler.fit_transform(df_normalized[numeric_cols])
            self.is_fitted = True
        else:
            if not self.is_fitted:
                raise ValueError("Scaler nie został dopasowany. Użyj fit=True dla danych treningowych.")
            df_normalized[numeric_cols] = self.scaler.transform(df_normalized[numeric_cols])
        
        return df_normalized
    
    def preprocess_pipeline(self, df: pd.DataFrame, target_col: str = None, 
                           handle_missing: str = 'drop', encode_method: str = 'label',
                           normalize: bool = True, fit: bool = True) -> tuple:
        """
        Pełny pipeline preprocessingu
        
        Parameters:
        -----------
        df : pd.DataFrame
            DataFrame do przetworzenia
        target_col : str
            Nazwa kolumny docelowej (jeśli None, nie separuje)
        handle_missing : str
            Strategia obsługi brakujących wartości
        encode_method : str
            Metoda kodowania kategorycznych
        normalize : bool
            Czy normalizować zmienne
        fit : bool
            Czy dopasować transformatory (True dla train)
            
        Returns:
        --------
        tuple
            (X, y) lub (df_processed, None) jeśli target_col=None
        """
        df_processed = df.copy()
        
        # Obsługa brakujących wartości
        df_processed = self.handle_missing_values(df_processed, strategy=handle_missing)
        
        # Separacja targetu jeśli podany
        if target_col and target_col in df_processed.columns:
            y = df_processed[target_col].copy()
            X = df_processed.drop(columns=[target_col])
        else:
            X = df_processed
            y = None
        
        # Kodowanie kategorycznych
        X = self.encode_categorical(X, method=encode_method)
        
        # Normalizacja
        if normalize:
            X = self.normalize_features(X, fit=fit)
        
        if y is not None:
            return X, y
        else:
            return X, None


In [ ]:
class DataAnalyzer:
    """Klasa odpowiedzialna za analizę danych i wizualizacje"""
    
    def __init__(self):
        pass
    
    def descriptive_statistics(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Zwraca statystyki opisowe
        
        Parameters:
        -----------
        df : pd.DataFrame
            DataFrame do analizy
            
        Returns:
        --------
        pd.DataFrame
            Statystyki opisowe
        """
        return df.describe()
    
    def correlation_analysis(self, df: pd.DataFrame, target: str = None) -> pd.DataFrame:
        """
        Analiza korelacji
        
        Parameters:
        -----------
        df : pd.DataFrame
            DataFrame do analizy
        target : str
            Nazwa kolumny docelowej
            
        Returns:
        --------
        pd.DataFrame
            Macierz korelacji
        """
        if target and target in df.columns:
            correlations = df.corr()[target].sort_values(ascending=False)
            return correlations
        else:
            return df.corr()
    
    def visualize_distributions(self, df: pd.DataFrame, columns: list = None, figsize: tuple = (15, 10)):
        """
        Wizualizuje rozkłady zmiennych numerycznych
        
        Parameters:
        -----------
        df : pd.DataFrame
            DataFrame do wizualizacji
        columns : list
            Lista kolumn do wizualizacji (None = wszystkie numeryczne)
        figsize : tuple
            Rozmiar wykresu
        """
        if columns is None:
            numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        else:
            numeric_cols = columns
        
        n_cols = len(numeric_cols)
        n_rows = (n_cols + 2) // 3
        
        fig, axes = plt.subplots(n_rows, 3, figsize=figsize)
        axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
        
        for idx, col in enumerate(numeric_cols):
            if idx < len(axes):
                df[col].hist(bins=30, ax=axes[idx], edgecolor='black')
                axes[idx].set_title(f'Rozkład {col}')
                axes[idx].set_xlabel(col)
                axes[idx].set_ylabel('Częstość')
        
        # Ukryj puste subploty
        for idx in range(len(numeric_cols), len(axes)):
            axes[idx].set_visible(False)
        
        plt.tight_layout()
        plt.show()
    
    def visualize_correlations(self, df: pd.DataFrame, figsize: tuple = (12, 10)):
        """
        Wizualizuje macierz korelacji
        
        Parameters:
        -----------
        df : pd.DataFrame
            DataFrame do wizualizacji
        figsize : tuple
            Rozmiar wykresu
        """
        numeric_df = df.select_dtypes(include=[np.number])
        corr_matrix = numeric_df.corr()
        
        plt.figure(figsize=figsize)
        sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                   center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
        plt.title('Macierz korelacji')
        plt.tight_layout()
        plt.show()
    
    def class_balance_analysis(self, y: pd.Series) -> dict:
        """
        Analiza balansu klas
        
        Parameters:
        -----------
        y : pd.Series
            Zmienna docelowa
            
        Returns:
        --------
        dict
            Słownik z informacjami o balansie klas
        """
        value_counts = y.value_counts()
        percentages = y.value_counts(normalize=True) * 100
        
        analysis = {
            "counts": value_counts.to_dict(),
            "percentages": percentages.to_dict(),
            "is_balanced": (percentages.min() > 40) and (percentages.max() < 60)
        }
        
        # Wizualizacja
        plt.figure(figsize=(8, 5))
        value_counts.plot(kind='bar', color=['skyblue', 'salmon'])
        plt.title('Rozkład klas docelowych')
        plt.xlabel('Klasa')
        plt.ylabel('Liczba obserwacji')
        plt.xticks(rotation=0)
        for i, v in enumerate(value_counts.values):
            plt.text(i, v, str(v), ha='center', va='bottom')
        plt.tight_layout()
        plt.show()
        
        return analysis


In [ ]:
class FeatureEngineer:
    """Klasa odpowiedzialna za feature engineering"""
    
    def __init__(self):
        self.feature_importance = None
    
    def create_interaction_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Tworzy cechy interakcyjne
        
        Parameters:
        -----------
        df : pd.DataFrame
            DataFrame do przetworzenia
            
        Returns:
        --------
        pd.DataFrame
            DataFrame z nowymi cechami
        """
        df_new = df.copy()
        
        # TotalPages = suma wszystkich stron
        if all(col in df_new.columns for col in ['Administrative', 'Informational', 'ProductRelated']):
            df_new['TotalPages'] = (df_new['Administrative'] + 
                                   df_new['Informational'] + 
                                   df_new['ProductRelated'])
        
        # TotalDuration = suma wszystkich czasów
        if all(col in df_new.columns for col in ['Administrative_Duration', 
                                                  'Informational_Duration', 
                                                  'ProductRelated_Duration']):
            df_new['TotalDuration'] = (df_new['Administrative_Duration'] + 
                                       df_new['Informational_Duration'] + 
                                       df_new['ProductRelated_Duration'])
        
        # AvgPageDuration = średni czas na stronę
        if 'TotalDuration' in df_new.columns and 'TotalPages' in df_new.columns:
            df_new['AvgPageDuration'] = df_new['TotalDuration'] / (df_new['TotalPages'] + 1e-6)
        
        # BounceExitRatio = stosunek bounce do exit
        if all(col in df_new.columns for col in ['BounceRates', 'ExitRates']):
            df_new['BounceExitRatio'] = df_new['BounceRates'] / (df_new['ExitRates'] + 1e-6)
        
        # ProductRelatedRatio = stosunek stron produktowych do wszystkich
        if 'ProductRelated' in df_new.columns and 'TotalPages' in df_new.columns:
            df_new['ProductRelatedRatio'] = df_new['ProductRelated'] / (df_new['TotalPages'] + 1e-6)
        
        return df_new
    
    def create_aggregated_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Tworzy cechy zagregowane
        
        Parameters:
        -----------
        df : pd.DataFrame
            DataFrame do przetworzenia
            
        Returns:
        --------
        pd.DataFrame
            DataFrame z nowymi cechami
        """
        df_new = df.copy()
        
        # Można dodać więcej cech zagregowanych tutaj
        # Na przykład: średnie, mediany, itp.
        
        return df_new
    
    def select_features(self, X: pd.DataFrame, y: pd.Series, 
                       method: str = 'correlation', threshold: float = 0.01) -> tuple:
        """
        Selekcja zmiennych
        
        Parameters:
        -----------
        X : pd.DataFrame
            Cechy
        y : pd.Series
            Zmienna docelowa
        method : str
            Metoda selekcji ('correlation', 'importance')
        threshold : float
            Próg dla selekcji
            
        Returns:
        --------
        tuple
            (X_selected, selected_features)
        """
        if method == 'correlation':
            correlations = X.corrwith(y).abs()
            selected_features = correlations[correlations >= threshold].index.tolist()
            X_selected = X[selected_features]
        
        elif method == 'importance':
            # Użyj Random Forest do określenia ważności cech
            rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
            rf.fit(X, y)
            
            feature_importance = pd.Series(rf.feature_importances_, index=X.columns)
            self.feature_importance = feature_importance.sort_values(ascending=False)
            
            selected_features = feature_importance[feature_importance >= threshold].index.tolist()
            X_selected = X[selected_features]
        
        else:
            X_selected = X
            selected_features = X.columns.tolist()
        
        return X_selected, selected_features


In [ ]:
class ModelTrainer:
    """Klasa odpowiedzialna za trenowanie modeli"""
    
    def __init__(self):
        self.models = {}
        self.results = {}
    
    def train_model(self, X_train, y_train, model_type: str, **kwargs):
        """
        Trenuje model
        
        Parameters:
        -----------
        X_train : array-like
            Cechy treningowe
        y_train : array-like
            Zmienna docelowa treningowa
        model_type : str
            Typ modelu ('logistic', 'random_forest', 'svm', 'xgboost')
        **kwargs
            Dodatkowe parametry modelu
            
        Returns:
        --------
        model
            Wytrenowany model
        """
        if model_type == 'logistic':
            model = LogisticRegression(random_state=42, max_iter=1000, **kwargs)
        elif model_type == 'random_forest':
            model = RandomForestClassifier(random_state=42, n_jobs=-1, **kwargs)
        elif model_type == 'svm':
            model = SVC(random_state=42, probability=True, **kwargs)
        elif model_type == 'xgboost':
            try:
                import xgboost as xgb
                model = xgb.XGBClassifier(random_state=42, **kwargs)
            except ImportError:
                print("XGBoost nie jest zainstalowany. Używam Random Forest zamiast tego.")
                model = RandomForestClassifier(random_state=42, n_jobs=-1, **kwargs)
        else:
            raise ValueError(f"Nieznany typ modelu: {model_type}")
        
        model.fit(X_train, y_train)
        self.models[model_type] = model
        
        return model
    
    def evaluate_model(self, model, X_test, y_test) -> dict:
        """
        Ewaluuje model
        
        Parameters:
        -----------
        model : model
            Model do ewaluacji
        X_test : array-like
            Cechy testowe
        y_test : array-like
            Zmienna docelowa testowa
            
        Returns:
        --------
        dict
            Słownik z metrykami
        """
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, zero_division=0),
            'recall': recall_score(y_test, y_pred, zero_division=0),
            'f1_score': f1_score(y_test, y_pred, zero_division=0),
            'roc_auc': roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None,
            'confusion_matrix': confusion_matrix(y_test, y_pred)
        }
        
        return metrics
    
    def compare_models(self, models: dict, X_test, y_test) -> pd.DataFrame:
        """
        Porównuje wiele modeli
        
        Parameters:
        -----------
        models : dict
            Słownik modeli {nazwa: model}
        X_test : array-like
            Cechy testowe
        y_test : array-like
            Zmienna docelowa testowa
            
        Returns:
        --------
        pd.DataFrame
            DataFrame z wynikami porównania
        """
        results = []
        
        for name, model in models.items():
            metrics = self.evaluate_model(model, X_test, y_test)
            metrics['model'] = name
            results.append(metrics)
        
        results_df = pd.DataFrame(results)
        results_df = results_df.set_index('model')
        
        return results_df


In [ ]:
class HyperparameterTuner:
    """Klasa odpowiedzialna za optymalizację hiperparametrów"""
    
    def __init__(self):
        self.best_params = {}
        self.best_scores = {}
    
    def grid_search(self, model, param_grid: dict, X_train, y_train, 
                   cv: int = 5, scoring: str = 'f1', n_jobs: int = -1):
        """
        Grid Search dla optymalizacji hiperparametrów
        
        Parameters:
        -----------
        model : model
            Model do optymalizacji
        param_grid : dict
            Siatka parametrów
        X_train : array-like
            Cechy treningowe
        y_train : array-like
            Zmienna docelowa treningowa
        cv : int
            Liczba foldów cross-validation
        scoring : str
            Metryka do optymalizacji
        n_jobs : int
            Liczba równoległych zadań
            
        Returns:
        --------
        model
            Najlepszy model
        """
        grid_search = GridSearchCV(
            model, 
            param_grid, 
            cv=cv, 
            scoring=scoring, 
            n_jobs=n_jobs,
            verbose=1
        )
        
        grid_search.fit(X_train, y_train)
        
        self.best_params[type(model).__name__] = grid_search.best_params_
        self.best_scores[type(model).__name__] = grid_search.best_score_
        
        print(f"Najlepsze parametry: {grid_search.best_params_}")
        print(f"Najlepszy wynik CV: {grid_search.best_score_:.4f}")
        
        return grid_search.best_estimator_
    
    def random_search(self, model, param_distributions: dict, X_train, y_train,
                     n_iter: int = 50, cv: int = 5, scoring: str = 'f1', n_jobs: int = -1):
        """
        Random Search dla optymalizacji hiperparametrów
        
        Parameters:
        -----------
        model : model
            Model do optymalizacji
        param_distributions : dict
            Rozkłady parametrów
        X_train : array-like
            Cechy treningowe
        y_train : array-like
            Zmienna docelowa treningowa
        n_iter : int
            Liczba iteracji
        cv : int
            Liczba foldów cross-validation
        scoring : str
            Metryka do optymalizacji
        n_jobs : int
            Liczba równoległych zadań
            
        Returns:
        --------
        model
            Najlepszy model
        """
        random_search = RandomizedSearchCV(
            model,
            param_distributions,
            n_iter=n_iter,
            cv=cv,
            scoring=scoring,
            n_jobs=n_jobs,
            random_state=42,
            verbose=1
        )
        
        random_search.fit(X_train, y_train)
        
        self.best_params[type(model).__name__] = random_search.best_params_
        self.best_scores[type(model).__name__] = random_search.best_score_
        
        print(f"Najlepsze parametry: {random_search.best_params_}")
        print(f"Najlepszy wynik CV: {random_search.best_score_:.4f}")
        
        return random_search.best_estimator_


## 3. Etap 1: Pobranie Danych


In [ ]:
# Inicjalizacja DataLoader
loader = DataLoader()

# Wczytanie danych
data = loader.load_data('online_shoppers_intention.csv')

# Podstawowe informacje o zbiorze
info = loader.get_info()
print("\n=== Podstawowe informacje o zbiorze ===")
print(f"Kształt: {info['shape']}")
print(f"\nKolumny ({len(info['columns'])}):")
for col in info['columns']:
    print(f"  - {col}")

print(f"\nBrakujące wartości:")
missing = {k: v for k, v in info['missing_values'].items() if v > 0}
if missing:
    for col, count in missing.items():
        print(f"  - {col}: {count}")
else:
    print("  Brak brakujących wartości!")

# Podgląd danych
print("\n=== Pierwsze 5 wierszy ===")
data.head()


In [ ]:
# Inicjalizacja preprocessora
preprocessor = DataPreprocessor()

# Pełny pipeline preprocessingu
X, y = preprocessor.preprocess_pipeline(
    data, 
    target_col='Revenue',
    handle_missing='drop',
    encode_method='label',
    normalize=True,
    fit=True
)

print(f"Kształt X: {X.shape}")
print(f"Kształt y: {y.shape}")
print(f"\nRozkład klas docelowych:")
print(y.value_counts())
print(f"\nProcentowy rozkład:")
print(y.value_counts(normalize=True) * 100)


In [ ]:
# Inicjalizacja analizatora
analyzer = DataAnalyzer()

# Statystyki opisowe
print("=== Statystyki opisowe ===")
stats = analyzer.descriptive_statistics(data.select_dtypes(include=[np.number]))
print(stats)


In [ ]:
# Analiza korelacji z targetem
print("=== Korelacje ze zmienną docelową (Revenue) ===")
correlations = analyzer.correlation_analysis(data, target='Revenue')
print(correlations.sort_values(ascending=False))


In [ ]:
# Wizualizacja macierzy korelacji
analyzer.visualize_correlations(data)


In [ ]:
# Analiza balansu klas
balance_info = analyzer.class_balance_analysis(y)
print(f"\nBalans klas: {'Zbalansowany' if balance_info['is_balanced'] else 'Niezbalansowany'}")


In [ ]:
# Wizualizacja rozkładów wybranych zmiennych
numeric_cols = ['Administrative', 'Administrative_Duration', 'ProductRelated', 
                'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues']
analyzer.visualize_distributions(data[numeric_cols + ['Revenue']], columns=numeric_cols)


## 6. Etap 4: Feature Engineering


In [ ]:
# Przygotowanie danych przed feature engineering (bez normalizacji)
X_raw, y_raw = preprocessor.preprocess_pipeline(
    data,
    target_col='Revenue',
    handle_missing='drop',
    encode_method='label',
    normalize=False,  # Nie normalizujemy jeszcze
    fit=True
)

# Inicjalizacja feature engineer
feature_engineer = FeatureEngineer()

# Tworzenie nowych cech
X_with_features = feature_engineer.create_interaction_features(X_raw)
X_with_features = feature_engineer.create_aggregated_features(X_with_features)

print(f"Liczba cech przed feature engineering: {X_raw.shape[1]}")
print(f"Liczba cech po feature engineering: {X_with_features.shape[1]}")
print(f"\nNowe cechy:")
new_features = set(X_with_features.columns) - set(X_raw.columns)
for feat in new_features:
    print(f"  - {feat}")


In [ ]:
# Selekcja zmiennych na podstawie ważności (Random Forest)
X_selected, selected_features = feature_engineer.select_features(
    X_with_features, 
    y_raw, 
    method='importance', 
    threshold=0.01
)

print(f"Liczba wybranych cech: {len(selected_features)}")
print(f"\nWybrane cechy:")
for feat in selected_features:
    print(f"  - {feat}")

# Wizualizacja ważności cech
if feature_engineer.feature_importance is not None:
    plt.figure(figsize=(12, 8))
    top_features = feature_engineer.feature_importance.head(20)
    top_features.plot(kind='barh', color='steelblue')
    plt.title('Top 20 najważniejszych cech (Feature Importance)')
    plt.xlabel('Ważność')
    plt.tight_layout()
    plt.show()


In [ ]:
# Normalizacja wybranych cech
X_final = preprocessor.normalize_features(X_selected, fit=True)

print(f"Końcowy kształt danych: {X_final.shape}")
print(f"Liczba obserwacji: {X_final.shape[0]}")
print(f"Liczba cech: {X_final.shape[1]}")


## 7. Etap 5: Przygotowanie Zbiorów


In [ ]:
# Podział na zbiór treningowy i testowy (80/20) z stratyfikacją
X_train, X_test, y_train, y_test = train_test_split(
    X_final, 
    y_raw, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_raw
)

print(f"Zbiór treningowy: {X_train.shape[0]} obserwacji")
print(f"Zbiór testowy: {X_test.shape[0]} obserwacji")
print(f"\nRozkład klas w zbiorze treningowym:")
print(y_train.value_counts(normalize=True) * 100)
print(f"\nRozkład klas w zbiorze testowym:")
print(y_test.value_counts(normalize=True) * 100)


## 8. Etap 6: Trenowanie Modeli


In [ ]:
# Inicjalizacja trenera modeli
trainer = ModelTrainer()

# Trenowanie różnych modeli
models = {}

print("=== Trenowanie Logistic Regression ===")
models['Logistic Regression'] = trainer.train_model(
    X_train, y_train, 
    model_type='logistic'
)

print("\n=== Trenowanie Random Forest ===")
models['Random Forest'] = trainer.train_model(
    X_train, y_train,
    model_type='random_forest',
    n_estimators=100
)

print("\n=== Trenowanie SVM ===")
models['SVM'] = trainer.train_model(
    X_train, y_train,
    model_type='svm',
    kernel='rbf',
    C=1.0
)

print("\n=== Trenowanie XGBoost (lub Random Forest jeśli XGBoost niedostępny) ===")
models['XGBoost'] = trainer.train_model(
    X_train, y_train,
    model_type='xgboost',
    n_estimators=100
)


In [ ]:
# Porównanie modeli
print("=== Porównanie modeli ===")
comparison = trainer.compare_models(models, X_test, y_test)
print(comparison)

# Wizualizacja porównania
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Wykres słupkowy metryk
metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1_score']
comparison[metrics_to_plot].plot(kind='bar', ax=axes[0], color=['skyblue', 'salmon', 'lightgreen', 'orange'])
axes[0].set_title('Porównanie metryk modeli')
axes[0].set_xlabel('Model')
axes[0].set_ylabel('Wartość')
axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].tick_params(axis='x', rotation=45)

# Tabela z wynikami
axes[1].axis('tight')
axes[1].axis('off')
table = axes[1].table(cellText=comparison.round(4).values,
                     rowLabels=comparison.index,
                     colLabels=comparison.columns,
                     cellLoc='center',
                     loc='center')
table.auto_set_font_size(False)
table.set_fontsize(9)
axes[1].set_title('Tabela wyników', pad=20)

plt.tight_layout()
plt.show()


## 9. Etap 7: Fine-tuning (Optymalizacja Hiperparametrów)


In [ ]:
# Wybór najlepszego modelu do tuningu (na podstawie wyników)
best_model_name = comparison['f1_score'].idxmax()
print(f"Najlepszy model przed tuningiem: {best_model_name}")
print(f"F1-score: {comparison.loc[best_model_name, 'f1_score']:.4f}")

# Inicjalizacja tunera
tuner = HyperparameterTuner()

# Optymalizacja dla Random Forest (zwykle najlepszy dla tego typu danych)
if best_model_name == 'Random Forest' or 'Random Forest' in models:
    print("\n=== Optymalizacja Random Forest ===")
    rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)
    
    param_grid_rf = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    best_rf = tuner.grid_search(
        rf_base,
        param_grid_rf,
        X_train, y_train,
        cv=5,
        scoring='f1'
    )
    
    models['Random Forest (Tuned)'] = best_rf


In [ ]:
# Porównanie przed i po tuningu
print("=== Porównanie przed i po tuningu ===")
rf_original_metrics = trainer.evaluate_model(models['Random Forest'], X_test, y_test)
rf_tuned_metrics = trainer.evaluate_model(models['Random Forest (Tuned)'], X_test, y_test)

comparison_tuning = pd.DataFrame({
    'Przed tuningiem': rf_original_metrics,
    'Po tuningu': rf_tuned_metrics
}).T

print(comparison_tuning[['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']])

# Wizualizacja
fig, ax = plt.subplots(figsize=(10, 6))
metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
x = np.arange(len(metrics))
width = 0.35

ax.bar(x - width/2, comparison_tuning.loc['Przed tuningiem', metrics], width, 
       label='Przed tuningiem', color='skyblue')
ax.bar(x + width/2, comparison_tuning.loc['Po tuningu', metrics], width, 
       label='Po tuningu', color='salmon')

ax.set_xlabel('Metryki')
ax.set_ylabel('Wartość')
ax.set_title('Porównanie modelu przed i po optymalizacji hiperparametrów')
ax.set_xticks(x)
ax.set_xticklabels(metrics, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


## 10. Etap 8: Ewaluacja - Szczegółowa Analiza Najlepszego Modelu


In [ ]:
# Wybór najlepszego modelu (po tuningu)
final_model = models['Random Forest (Tuned)']
final_metrics = trainer.evaluate_model(final_model, X_test, y_test)

print("=== Szczegółowe metryki najlepszego modelu ===")
print(f"Accuracy: {final_metrics['accuracy']:.4f}")
print(f"Precision: {final_metrics['precision']:.4f}")
print(f"Recall: {final_metrics['recall']:.4f}")
print(f"F1-score: {final_metrics['f1_score']:.4f}")
print(f"ROC-AUC: {final_metrics['roc_auc']:.4f}")

# Confusion Matrix
cm = final_metrics['confusion_matrix']
print(f"\nConfusion Matrix:")
print(cm)

# Wizualizacja Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Purchase', 'Purchase'],
            yticklabels=['No Purchase', 'Purchase'])
plt.title('Confusion Matrix - Random Forest (Tuned)')
plt.ylabel('Rzeczywistość')
plt.xlabel('Przewidywanie')
plt.tight_layout()
plt.show()

# Classification Report
y_pred_final = final_model.predict(X_test)
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred_final, 
                          target_names=['No Purchase', 'Purchase']))


In [ ]:
# ROC Curve
y_pred_proba = final_model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'ROC curve (AUC = {final_metrics["roc_auc"]:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Random Forest (Tuned)')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Feature Importance dla najlepszego modelu
if hasattr(final_model, 'feature_importances_'):
    feature_importance_df = pd.DataFrame({
        'feature': X_final.columns,
        'importance': final_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    top_20 = feature_importance_df.head(20)
    plt.barh(range(len(top_20)), top_20['importance'], color='steelblue')
    plt.yticks(range(len(top_20)), top_20['feature'])
    plt.xlabel('Ważność cechy')
    plt.title('Top 20 najważniejszych cech - Random Forest (Tuned)')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\n=== Top 10 najważniejszych cech ===")
    print(feature_importance_df.head(10))


In [ ]:
# Cross-validation dla ostatecznej weryfikacji
print("=== Cross-Validation (5-fold) ===")
cv_scores = cross_val_score(final_model, X_train, y_train, cv=5, scoring='f1')
print(f"F1-scores dla każdego folda: {cv_scores}")
print(f"Średni F1-score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Wizualizacja CV scores
plt.figure(figsize=(8, 5))
plt.bar(range(1, 6), cv_scores, color='lightblue', edgecolor='black')
plt.axhline(y=cv_scores.mean(), color='red', linestyle='--', 
           label=f'Średnia: {cv_scores.mean():.4f}')
plt.xlabel('Fold')
plt.ylabel('F1-score')
plt.title('Cross-Validation Scores (5-fold)')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


## 11. Podsumowanie i Wnioski

### Wyniki:
- **Najlepszy model**: Random Forest z optymalizacją hiperparametrów
- **Accuracy**: {final_metrics['accuracy']:.4f}
- **F1-score**: {final_metrics['f1_score']:.4f}
- **ROC-AUC**: {final_metrics['roc_auc']:.4f}

### Analiza błędów:
- **False Positives**: {cm[0][1]} - Model przewiduje zakup, ale użytkownik nie kupił
- **False Negatives**: {cm[1][0]} - Model nie przewiduje zakupu, ale użytkownik kupił

### Pomysły na dalszy rozwój:
1. Przetestowanie innych algorytmów (Gradient Boosting, Neural Networks)
2. Zbalansowanie zbioru danych (SMOTE, undersampling)
3. Głębsza analiza feature engineering
4. Ensemble methods (łączenie wielu modeli)
5. Analiza błędnych predykcji w celu zrozumienia wzorców
